# Exercise - Multi step Workflow - STARTER

In this exercise, you’ll build a multi-step workflow using LCEL to solve a more complex task than simply generating a joke. 

**Challenge**

Create an AI Business Advisor that:

1. Accepts an industry as input.
2. Generates a business idea.
3. Analyzes the strengths and weaknesses.
4. Formats the results as a final report.


## 0. Import the necessary libs

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")
base_url = os.getenv("OPENAI_ENDPOINT")
model_name = "gpt-4o-mini"
temp=0.0

llm = ChatOpenAI(
    base_url=base_url,
    api_key=api_key,
    model=model_name,
    temperature=temp
)

## 1. Instantiate Chat Model with your API Key

To be able to connect with OpenAI, you need to instantiate an ChatOpenAI client passing your OpenAI key.


## 2. Your first Chain

In the end of each chain, you should parse the output and save the logs

In [ ]:
logs = []
parser = StrOutputParser()
parse_and_log_output_chain = RunnableParallel(
    output=parser, 
    log=RunnableLambda(lambda x: logs.append(x))
)

## 3. Idea Generation

Craft a prompt to generate a business idea for the given industry. 

Make sure {industry} placeholder is inside your template, so it can be filled when the chain is invoked.

In [ ]:
# TODO - Your prompt Template
idea_prompt = PromptTemplate(
    template=(
        """
You are a startup founder and Generative AI product expert.

Generate one AI-first startup business idea for the {industry} industry.

Your response must include:

Idea Summary – What the product is and the core problem it solves

Target Users – Who will use it and why they care

AI Advantage – How AI is central (models, automation, data, agents, etc.)

Differentiation – Why this beats existing solutions

Business Model – How the startup makes money

MVP Scope – What can be built in 60–90 days

Scaling Path – How this grows after product-market fit

Keep the output concise, concrete, and execution-oriented, as if pitching to early-stage investors. Avoid generic ideas; focus on AI-native leverage and real customer pain.
        
        """
    )
)

In [ ]:
# TODO - Create your idea_chain: idea_prompt -> llm -> parse_and_log_output_chain
idea_chain = idea_prompt | llm | parse_and_log_output_chain

In [ ]:
# TODO - Test your idea_chain invoking it by passing an industy like "agro" to it
idea_result = idea_chain.invoke({"industry": "Real Estate"})

In [ ]:
idea_result["output"] # string

In [ ]:
print(idea_result["output"])

In [ ]:
logs

## 4. Idea Analysis

Craft a prompt to analyze the generated idea's strengths and weaknesses

In [ ]:
# TODO - Your prompt Template
analysis_prompt = PromptTemplate(
    template=(
        """
You are a startup advisor and AI product strategist.

Analyze the following business idea concisely and critically.

Business Idea:
{business_idea}

Provide:

Strengths – Clear value, AI leverage, market pain, differentiation

Weaknesses & Risks – Product, data, adoption, competition

Execution Concerns – MVP feasibility, skills, cost, timeline

Defensibility – Ease of copying and moat potential

Verdict – Proceed, Pivot, or Kill with 1–2 improvement suggestions

Keep the analysis brief, blunt, and practical, as if advising a seed-stage founder or investor.
        """
    )
)

In [ ]:
# TODO - Your chain
analysis_chain = analysis_prompt | llm | parse_and_log_output_chain

In [ ]:
# TODO - Test your analysis_chain invoking it by passing idea_result["output"] to it
analysis_result = analysis_chain.invoke({"business_idea": idea_result})

In [ ]:
print(analysis_result["output"])

In [ ]:
logs

## 5. Report Generation

Craft a prompt to structure the information into a business report.

In [ ]:
# TODO - Your prompt Template
report_prompt = PromptTemplate(
    template=(
        "" 
    )
)

In [ ]:
class AnalysisReport(BaseModel):
    """Strengths and Weaknesses about a business idea"""
    strengths: list = Field(default=[], description="Idea's strength list")
    weaknesses: list = Field(default=[], description="Idea's weaknesse list")

In [ ]:
report_chain = (
    report_prompt | llm.with_structured_output(AnalysisReport)
)

In [ ]:
# TODO - Test your report_chain invoking it by passing analysis_result["output"] to it
report_result = 

In [ ]:
report_result

## 6. End to End Chain

In [ ]:
e2e_chain = ( 
    RunnablePassthrough() 
    | idea_chain
    | RunnableParallel(idea=RunnablePassthrough())
    | analysis_chain
    | report_chain
)

In [ ]:
e2e_chain.get_graph().print_ascii()

In [ ]:
# Change the industry if you want
e2e_result = e2e_chain.invoke("agro")

In [ ]:
e2e_result

In [ ]:
e2e_result.strengths

In [ ]:
e2e_result.weaknesses

In [ ]:
logs

## 7. Experiment

Now that you understood how it works, experiment with new things.
- Improve memory
- Explore the Runnables
- Add More Complexity